# GROMACS Molecular Dynamics Workflow

This document contains a step-by-step guide to setting up, running, and analyzing molecular dynamics (MD) simulations in GROMACS. It includes all the commands used, their functions, and updated input files (ions.mdp, nvt.mdp, npt.mdp, md.mdp) for future reference.

---

## **Workflow Overview**
1. **Protein Preparation**
2. **Solvation**
3. **Adding Ions**
4. **Energy Minimization**
5. **NVT (Constant Volume and Temperature) Equilibration**
6. **NPT (Constant Pressure and Temperature) Equilibration**
7. **Production Simulation**

---

## **1. Protein Preparation**
Convert the PDB file of your protein into a GROMACS-compatible format and generate topology files.

```bash
gmx pdb2gmx -f protein.pdb -o protein_processed.gro -water tip3p
```
- **`-f protein.pdb`**: Input protein structure.
- **`-o protein_processed.gro`**: Output processed structure.
- **`-water tip3p`**: Specifies the TIP3P water model.

---

## **2. Solvation**
Define the simulation box and fill it with water molecules.

```bash
gmx editconf -f protein_processed.gro -o boxed.gro -c -d 1.0 -bt cubic
```
- **`-c`**: Centers the protein in the box.
- **`-d 1.0`**: Adds 1 nm between the protein and box edges.
- **`-bt cubic`**: Defines a cubic box.

```bash
gmx solvate -cp boxed.gro -cs spc216.gro -o solvated.gro -p topol.top
```
- **`-cp boxed.gro`**: Input box file.
- **`-cs spc216.gro`**: Water model file.
- **`-p topol.top`**: Updates topology file to include water molecules.

---

## **3. Adding Ions**
Neutralize the system and adjust the ion concentration.

### **Ions.mdp**
```plaintext
integrator      = steep
emtol           = 1000.0
emstep          = 0.01
nsteps          = 50000
nstenergy       = 1
energygrps      = System
cutoff-scheme   = Verlet
nstlist         = 10
ns_type         = grid
rlist           = 1.0
rcoulomb        = 1.0
rvdw            = 1.0
pbc             = xyz
```

### Commands:
1. **Prepare TPR for ion addition:**
```bash
gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr
```
2. **Add ions:**
```bash
gmx genion -s ions.tpr -o solvated_ions.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15
```
- **`-neutral`**: Neutralizes the system.
- **`-conc 0.15`**: Adds 150 mM NaCl.

---

## **4. Energy Minimization**
Minimize the system's energy to remove clashes.

### **Minimization Command:**
```bash
gmx grompp -f minim.mdp -c solvated_ions.gro -p topol.top -o em.tpr
gmx mdrun -deffnm em
```
### **Minimization MDP Parameters:**
Use the provided `ions.mdp` file for energy minimization.

---

## **5. NVT Equilibration**
Stabilize temperature with position restraints.

### **NVT MDP:**
```plaintext
title                   = CHARMM27 Protein NVT equilibration
define                  = -DPOSRES
integrator              = md
nsteps                  = 50000
dt                      = 0.002
nstxout-compressed      = 500
tcoupl                  = V-rescale
tc-grps                 = Protein Water_and_ions
tau_t                   = 0.1 0.1
ref_t                   = 310 310
pbc                     = xyz
constraints             = h-bonds
```
### Commands:
1. **Prepare TPR for NVT:**
```bash
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
```
2. **Run NVT:**
```bash
gmx mdrun -deffnm nvt
```

---

## **6. NPT Equilibration**
Stabilize pressure and temperature.

### **NPT MDP:**
```plaintext
title                   = CHARMM27 Protein NPT equilibration
define                  = -DPOSRES
integrator              = md
nsteps                  = 50000
dt                      = 0.002
nstxout-compressed      = 500
tcoupl                  = V-rescale
tc-grps                 = Protein Water_and_ions
tau_t                   = 0.1 0.1
ref_t                   = 310 310
pcoupl                  = Parrinello-Rahman
pcoupltype              = isotropic
tau_p                   = 2.0
ref_p                   = 1.0
compressibility         = 4.5e-5
pbc                     = xyz
constraints             = h-bonds
```
### Commands:
1. **Prepare TPR for NPT:**
```bash
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -p topol.top -o npt.tpr
```
2. **Run NPT:**
```bash
gmx mdrun -deffnm npt
```

---

## **7. Production Simulation**
Run the final unrestrained MD simulation.

### **MD MDP:**
```plaintext
title                   = CHARMM27 Protein Production MD
integrator              = md
nsteps                  = 5000000
dt                      = 0.002
nstxout-compressed      = 5000
tcoupl                  = V-rescale
tc-grps                 = Protein Water_and_ions
tau_t                   = 0.1 0.1
ref_t                   = 310 310
pcoupl                  = Parrinello-Rahman
pcoupltype              = isotropic
tau_p                   = 2.0
ref_p                   = 1.0
compressibility         = 4.5e-5
pbc                     = xyz
constraints             = h-bonds
```
### Commands:
1. **Prepare TPR for MD:**
```bash
gmx grompp -f md.mdp -c npt.gro -p topol.top -o md.tpr
```
2. **Run Production Simulation:**
```bash
gmx mdrun -deffnm md
```

---

This document can be used as a template for future GROMACS simulations, ensuring a consistent and efficient workflow.

